In [1]:
import pandas as pd
import re

In [40]:
files = ['20-2850920_baylor-scott-and-white-continuing-care-hospital_standardcharges.csv',
 '20-3749695_baylor-scott-and-white-emergency-med-ctr-cedar-park_standardcharges.csv',
 '74-1166904_baylor-scott-and-white-mclane-childrens_standardcharges.csv',
 '81-3040663_baylor-scott-and-white-medical-center-austin_standardcharges.csv',
 '81-3040663_baylor-scott-and-white-medical-center-buda_standardcharges.csv',
 '74-2519752_baylor-scott-and-white-medical-center-brenham_standardcharges.csv',
 '27-4434451_baylor-scott-and-white-med-center-college-station_standardcharges.csv',
 '74-1161944_baylor-scott-and-white-medical-center-hillcrest_standardcharges.csv',
 '20-3749695_baylor-scott-and-white--medical-center-lakeway_standardcharges.csv',
 '46-4007700_baylor-scott-and-white-medical-center-marble-falls_standardcharges.csv',
 '81-3040663_baylor-scott-and-white-med-center-pflugerville_standardcharges.csv',
 '20-3749695_baylor-scott-and-white-medical-center-round-rock_standardcharges.csv',
 '74-1595711_baylor-scott-and-white-medical-center-taylor_standardcharges.csv',
 '74-1166904_baylor-scott-and-white-medical-center-temple_standardcharges.csv',
 '74-1166904_baylor-scott-and-white-pavilion-temple_standardcharges.csv',
 '75-2834135_baylor-scott-and-white-heart-and-vascular-hospital_standardcharges.csv',
 '75-1008430_baylor-scott-and-white-all-saints-medical-center_standardcharges.csv',
 '82-4052186_baylor-scott-and-white-medical-center-centennial_standardcharges.csv',
 '75-1777119_baylor-scott-and-white-medical-center-grapevine_standardcharges.csv',
 '75-2586857_baylor-scott-and-white-medical-center-irving_standardcharges.csv',
 '26-0194016_baylor-scott-and-white-medical-center-lake-pointe_standardcharges.csv',
 '75-1037591_baylor-scott-and-white-medical-center-mckinney_standardcharges.csv',
 '82-0551704_baylor-scott-and-white-medical-center-plano_standardcharges.csv',
 '75-1844139_baylor-scott-and-white-medical-center-waxahachie_standardcharges.csv',
 '26-3603862_baylor-scott-and-white-heart-hospital-denton_standardcharges.csv',
 '41-2101361_baylor-scott-and-white-heart-hospital-mckinney_standardcharges.csv',
 '41-2101361_baylor-scott-and-white-heart-hospital-plano_standardcharges.csv',
 '75-1837454_baylor-university-medical-center_standardcharges.csv']

28

In [41]:
df = pd.read_csv('hospital.csv', dtype=str)

In [38]:
for file in files:
    match = re.search(r'(?<=_)[a-z]+(-[a-z]+)*', file)
    if match:
        name = match.group(0).replace('-', ' ').upper()
        print(name)
        df['score'] = df['name'].apply(lambda x: sum([1 for word in name.split() if word in x]))
        max_score = df['score'].max()
        if max_score > 0:
            matches = df.loc[df['score'] == max_score]
            if len(matches) == 1:
                df.loc[df['score'] == max_score, 'file_name'] = file
            else:
                # If there are multiple matches with the same score, use the one with the longest name
                exact_matches = matches.loc[matches['name'] == name.replace('UNIVERSITY', '').strip()]
                if len(exact_matches) == 1:
                    df.loc[df['name'] == exact_matches.iloc[0]['name'], 'file_name'] = file
                else:
                    exact_matches = matches.loc[matches['name'].str.contains(name.replace('UNIVERSITY', '').strip(), regex=False)]
                    if len(exact_matches) == 1:
                        df.loc[df['name'] == exact_matches.iloc[0]['name'], 'file_name'] = file
                    else:
                        max_length = matches['name'].str.len().max()
                        df.loc[(df['score'] == max_score) & (df['name'].str.len() == max_length), 'file_name'] = file

# df[['name', 'file_name', 'city']].to_csv('test.csv', index=False)

BAYLOR SCOTT AND WHITE CONTINUING CARE HOSPITAL
BAYLOR SCOTT AND WHITE EMERGENCY MED CTR CEDAR PARK
BAYLOR SCOTT AND WHITE MCLANE CHILDRENS
BAYLOR SCOTT AND WHITE MEDICAL CENTER AUSTIN
BAYLOR SCOTT AND WHITE MEDICAL CENTER BUDA
BAYLOR SCOTT AND WHITE MEDICAL CENTER BRENHAM
BAYLOR SCOTT AND WHITE MED CENTER COLLEGE STATION
BAYLOR SCOTT AND WHITE MEDICAL CENTER HILLCREST
BAYLOR SCOTT AND WHITE
BAYLOR SCOTT AND WHITE MEDICAL CENTER MARBLE FALLS
BAYLOR SCOTT AND WHITE MED CENTER PFLUGERVILLE
BAYLOR SCOTT AND WHITE MEDICAL CENTER ROUND ROCK
BAYLOR SCOTT AND WHITE MEDICAL CENTER TAYLOR
BAYLOR SCOTT AND WHITE MEDICAL CENTER TEMPLE
BAYLOR SCOTT AND WHITE PAVILION TEMPLE
BAYLOR SCOTT AND WHITE HEART AND VASCULAR HOSPITAL
BAYLOR SCOTT AND WHITE ALL SAINTS MEDICAL CENTER
BAYLOR SCOTT AND WHITE MEDICAL CENTER CENTENNIAL
BAYLOR SCOTT AND WHITE MEDICAL CENTER GRAPEVINE
BAYLOR SCOTT AND WHITE MEDICAL CENTER IRVING
BAYLOR SCOTT AND WHITE MEDICAL CENTER LAKE POINTE
BAYLOR SCOTT AND WHITE MEDICAL CENTER

In [39]:
# df.to_csv('test.csv', index=False)

In [2]:
df = pd.read_csv('test.csv', dtype=str)

In [47]:
not_found_files = [f for f in files if not any(name in f for name in df['file_name'].values)]
not_found_files

['74-1166904_baylor-scott-and-white-mclane-childrens_standardcharges.csv',
 '74-1166904_baylor-scott-and-white-pavilion-temple_standardcharges.csv',
 '41-2101361_baylor-scott-and-white-heart-hospital-mckinney_standardcharges.csv']

In [3]:
df['ein'] = df['file_name'].str.split('_').str[0]

In [5]:
ein_ccn_dict = dict(zip(df['ein'], df['id']))
ein_ccn_dict

{'74-1166904': '450054',
 '75-2586857': '450079',
 '74-1161944': '450101',
 '75-1008430': '450137',
 '75-1844139': '450372',
 '75-1777119': '450563',
 '75-2834135': '450851',
 '82-0551704': '450890',
 '74-1595711': '451374',
 '20-2850920': '452105',
 '41-2101361': '670025',
 '20-3749695': '670043',
 '27-4434451': '670088',
 '46-4007700': '670108',
 '81-3040663': '670136',
 '74-2519752': '450187',
 '82-4052186': '450885',
 '26-0194016': '450742',
 '75-1037591': '670082',
 '26-3603862': '450893',
 '75-1837454': '450021'}

In [59]:
df['stdchg_file_url'] = 'https://www.bswhealth.com/-/media/project/bsw/sites/bswhealth/documents/patient-tools/estimate-cost-of-care/' + df['file_name']

In [6]:
df['stdchg_file_url']

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
5     NaN
6     NaN
7     NaN
8     NaN
9     NaN
10    NaN
11    NaN
12    NaN
13    NaN
14    NaN
15    NaN
16    NaN
17    NaN
18    NaN
19    NaN
20    NaN
21    NaN
22    NaN
23    NaN
24    NaN
Name: stdchg_file_url, dtype: object

In [61]:
df.to_csv('hospital_import.csv', index=False)